In [1]:
import sys
sys.path.append("../")
sys.path.append("../../../assets")

# Get rid of annoying tf warning
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import bayesflow as beef
import pandas as pd

from experiments import NonStationaryDDMExperiment
from models import RandomWalkDDM

/home/divergence/anaconda3/envs/bfPower/lib/python3.10/site-packages/bayesflow/trainers.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# gpu setting and checking
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Constants

In [3]:
TRAIN_NETWORKS = False

NUM_SAMPLES = 2000

PARAM_LABELS = ['Drift rate', 'Threshold', 'Non-decision time']
PARAM_NAMES  = [r'$v$', r'$a$', r'$\tau$']
FONT_SIZE_1 = 22
FONT_SIZE_2 = 20
FONT_SIZE_3 = 18

In [12]:
model = RandomWalkDDM()
experiment = NonStationaryDDMExperiment(model)

INFO:root:Performing 2 pilot runs with the random_walk_ddm model...
INFO:root:Shape of parameter batch after 2 pilot simulations: (batch_size = 2, 800, 3)
INFO:root:Shape of simulation batch after 2 pilot simulations: (batch_size = 2, 800)
INFO:root:Shape of hyper_prior_draws batch after 2 pilot simulations: (batch_size = 2, 3)
INFO:root:Shape of local_prior_draws batch after 2 pilot simulations: (batch_size = 2, 800, 3)
INFO:root:No shared_prior_draws provided.
INFO:root:No optional simulation batchable context provided.
INFO:root:No optional simulation non-batchable context provided.
INFO:root:No optional prior batchable context provided.
INFO:root:No optional prior non-batchable context provided.
INFO:root:Performing a consistency check with provided components...
INFO:root:Done.


In [13]:
experiment = NonStationaryDDMExperiment(model)

INFO:root:Performing a consistency check with provided components...
INFO:root:Done.


In [15]:
experiment.trainer

# Training

In [9]:
if TRAIN_NETWORKS:
    history = experiment.run(
        epochs=75, 
        iterations_per_epoch=1000, 
        batch_size=16
    )
else:
    history = experiment.trainer.loss_history.get_plottable()

In [11]:
history

{}

In [10]:
f = beef.diagnostics.plot_losses(history)

AttributeError: 'dict' object has no attribute 'columns'

# Evaluation

## Calibration Error

## Parameter Recovery

# Parameter Estimation

In [10]:
data = pd.read_csv(
    '../data/data_color_discrimination.csv',
    )
data['rt'][data['correct'] == 0] = -(data['rt'][data['correct'] == 0])
data.head()

/var/folders/bv/vht4zx991wd5ygpm2yz12gd40000gn/T/ipykernel_41914/4241371757.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['rt'][data['correct'] == 0] = -(data['rt'][data['correct'] == 0])


,id,tutorial,speed_condition,difficulty,rt,resp,correct_response,correct
0,1,0,1,1,-0.801806,a,l,0
1,1,0,1,1,0.639257,a,a,1
2,1,0,1,1,-0.373820,l,a,0
3,1,0,1,1,-0.517185,l,a,0
4,1,0,1,1,0.834339,a,a,1
